In [2]:
# Pkg used
using SparseArrays
using Arpack
using BenchmarkTools
using DelimitedFiles
using PyPlot

This is the full ED code for Kitaev-Heisernberg model on $2 \times 2 \times 3$ Kagome lattice, in which the periodic boundary condition is used.

\section{Kagome Lattice}

Kagome lattice is equvalent to replacing each site of a triangle lattice by a triangle. Therefore, it has three unequivalent sites in an unit cell. Here, we lable each atom as $(r,c,atom)$, where $(r,c)$ labels the position of an unit cell in the lattice, and $atom$ labels the a specific atom in the unit cell.

<img src = "Kagome-223.png" width = 200>

Note that we deliberately label the three atoms and three bonds like above so that there are only $k$-bond between $i,j$-atoms,where $i,j,k =1,2,3$ and $i \neq j \neq k$. This could simplify the code.

In [9]:
struct Lattice{T <: Integer}
    # Lattice size set up
    N1::T
    N2::T
end

struct Position{T <: Integer}
    # position of an atom in the lattice
    row::T
    col::T
    atom::T
end

dimension(la::Lattice) = 2^(la.N1 * la.N2 * 3)
sitenum(la::Lattice) = la.N1 * la.N2 * 3

function index(pos::Position, incell::Int, which::Int, la::Lattice)
    #= Calculate the index of the atoms in the (r,c) unit cell.
       c, l start from 0.
       A-sublattice atom = 0; B-subkattice: atom = 1; C-subkattice: atom = 2.
       incell = 0,1 corresponds atoms out or inside of the unitcell
       which = 0, 1, 2, 3 corresponds to the original atom and 
        the one linked to it via x,y,z bonds =#
    sgn = 0
    if which == pos.atom + 1
        return(println("ERROR: No such bond!"))
    elseif which == 0
        if incell == 1
            a = pos.atom
            c = pos.col % la.N1
            r = pos.row % la.N2
        elseif incell == 0
            return println("ERROR: directions should be 1,2,3.")
        end
    elseif which != pos.atom + 1
        a = 5 - (pos.atom + 1) - which
        if which == 1
            sgn = pos.atom - a
            c = (pos.col + sgn)% la.N1
            r = pos.row % la.N2
        elseif which == 2
            sgn = pos.atom + 1 - which
            c = pos.col% la.N1
            r = (pos.row + sgn)% la.N2
        elseif which == 3
            sgn = pos.atom - a
            c = (pos.col + sgn)% la.N1
            r = (pos.row - sgn)% la.N2
        end
    end
   
    c < 0 ? c += la.N1 : c += 0
    r < 0 ? r += la.N2 : r += 0
    
    #println(r, c, a)
    n = r * la.N1 + c
    return 3 * n + 1 + a
end

function findposition(ind::Int, la::Lattice)
    #= find the position of ind in the lattice
       ind starts from 1 =#
    atom = (ind -1)%3 
    n = div((ind -1 -atom),3)
    r, c = divrem(n, la.N1)
    return Position(r, c, atom)
end


findposition (generic function with 1 method)

\section{Hamiltonian}

Here we consider spin-1/2 Kitaev-Heisenberg model:

\begin{equation}
    H = \sum_{<i,j>_\alpha} K S_i^\alpha S_j^\alpha + \sum_{<i,j>,\alpha} J S_i^\alpha S_j^\alpha
\end{equation}

where $\alpha = x,y,z$. We introduce parametrization $(J,K) = (I \cos \theta, I \sin \theta)$, where $I$ is the energy unit. We can write the $S_i^\alpha S_j^\alpha$ as:

\begin{align}
      S_i^x S_j^x &= \frac{1}{4} ( S_i^+ S_j^- + S_i^- S_j^+ + S_i^+ S_j^+            + S_i^- S_j^-)\\
      S_i^y S_j^y &= \frac{1}{4} ( S_i^+ S_j^- + S_i^- S_j^+ - S_i^+ S_j^+            - S_i^- S_j^-)\\
      S_i^z S_j^z &= S_i^z S_j^z \\
\end{align}

where $S_i^\pm = S_i^x \pm i S_i^y$.


In [7]:
# Tool functions
function bits(i::Integer, num::Integer)
    # Chenck the i-th element of the binary representation of a dicimal number
    # the bits count from right to left
    mask = 2^(i - 1)
    if num & mask == mask
        return 1
    else
        return 0
    end
end

function flip(i::Int, j::Int, tag::Int)
    #=Flip the spin on i,j site.
    Inputs: tag: tag of a state
            i,j: position of spins that are flipped
    Output: The tag of new state, type: int =#
    mask = 2^(i-1) + 2^(j-1)
    return xor(tag, mask)
end

function append_data(j::Int, colptr::Int, count::Int, row::Array{Int,1})
    point = colptr - 1
    for pos = colptr: count
        point += 1
        if row[pos] == j
            return pos, count
            break
        end
    end
            
    if point == count
        return count, count+1
    end
end

append_data (generic function with 1 method)

In [70]:
function apply_Kitaev(ind::Int, bond::Int, incell::Int, si::Int, K::Array{Float64,1}, la::Lattice)
    # apply the ind-th Kitaev term on bond to si state
    pos = findposition(ind, la)
    if bond != pos.atom + 1
        nind = index(pos, incell, bond, la)
        val = 0.0
        if bond == 1
            sf = flip(ind, nind, si)
            val = K[1] /4
        elseif bond == 2
            sf = flip(ind, nind, si)
            val = -K[2] * (bits(ind, si) - 0.5) * (bits(nind, si) - 0.5)
        elseif bond == 3
            sf = si
            val = K[3] * (bits(ind, si) - 0.5) * (bits(nind, si) - 0.5)
        end
    elseif bond == pos.atom + 1
        sf = si
        val = 0.0
    end
    return sf, val
end

function apply_J(ind::Int, bond::Int, incell::Int, isz::Int, si::Int,J::Array{Float64,1}, la::Lattice)
    # Nearest neighbor Heisenberg Hamiltonian
    pos = findposition(ind, la)
    if bond != pos.atom + 1
        nind = index(pos, incell, bond, la)
        val = 0.0
        if isz == 1
            sf = si
            val = J[3] * (bits(ind, si) - 0.5) * (bits(nind, si) - 0.5)
        else
            sf = flip(ind, nind, si)
            val = J[1]/4 - J[2] * (bits(ind, si) - 0.5) * (bits(nind, si) - 0.5)
        end
    elseif bond == pos.atom + 1
        sf = si
        val = 0.0
    end
    return sf, val
end

apply_J (generic function with 2 methods)

In [85]:
function Kitaev(K::Array{Float64,1}, la::Lattice)
    dim = dimension(la)
    num = sitenum(la)
    max = dim * (num + 1)
    error = 1.e-8
    
    col, row, data = zeros(Int, dim+1), zeros(Int, max), zeros(max)
    colptr, count = 1, 1
    
    K = K/2
    
    for si = 0: (dim-1)
        col[si+1] = colptr
        for ind = 1:num, bond = 1:3, incell = 0:1
            sf, val = apply_Kitaev(ind, bond, incell, si, K, la)
            if abs(val) > error
                pos, count = append_data(sf + 1, colptr, count, row)
                row[pos] = sf + 1
                data[pos] += val
            end
        end
        colptr = count
    end
    col[dim + 1] = count
    H = SparseMatrixCSC(dim, dim, col, row[1:count-1], data[1:count-1])
    H = sparse(H')
    return (1/2)*(H + H')
end

function Heisenberg(J::Array{Float64,1}, la::Lattice)
    dim = dimension(la)
    num = sitenum(la)
    max = dim * (num + 1)
    error = 1.e-8
    
    col, row, data = zeros(Int, dim+1), zeros(Int, max), zeros(max)
    colptr, count = 1, 1
    
    J = J/2
    
    for si = 0: (dim - 1)
        col[si + 1] = colptr
        for ind = 1:num, bond = 1:3, isz = 0:1, incell = 0:1
            sf, val = apply_J(ind, bond, incell, isz, si, J, la)
            if abs(val) > error
                pos, count = append_data(sf + 1, colptr, count, row)
                row[pos] = sf + 1
                data[pos] += val
            end
        end
        colptr = count
    end
    col[dim + 1] = count
    H = SparseMatrixCSC(dim, dim, col, row[1:count-1], data[1:count-1])
    H = sparse(H')
    return (1/2)*(H + H')
end

Heisenberg (generic function with 2 methods)

In [84]:
K= [-1.,0.,0.]
la = Lattice(2,2)

Hh = Heisenberg(K, la)
e, x = eigs(Hh, which = :SR)
println(e/12)

[-0.5, -0.5, -0.5, -0.5, -0.5, -0.5]


\section{Specfic heat, Entropy and Static spin structure factor}

In this section, we calculate specfic heat $C(T)$, entropy $S(T)$ and the $z$-component of static spin structure factor(SSSF) $S_q^z$.

The partition function $Z(T)$ of the canonical ensemble at temperature $T$ is expressed as follows:
\begin{equation}
    Z(T) = {\rm Tr} e^{-\beta H}  
\end{equation}

If we have all eigen-states $\{ | \psi_i \rangle \}$ and eigen-energies $E_i$, then $Z(T)$ can be expressed as:

\begin{equation}
    Z(T) = \sum_{i=1}^{N_s} \langle \psi_i | e^{-\beta H} | \psi_i \rangle 
       = \sum_{i=1}^{N_s} e^{-\beta E_i}
\end{equation}

The energy $E(T)$, specific heat $C(T)$ and entropy $S(T)$ can be calculated using the following general functions:

\begin{equation}
    E(T) = - \frac{\partial}{\partial \beta} \ln Z(T) 
        = \frac{1}{Z(T）}\sum_{i=1}^{N_s} E_i e^{-\beta E_i}
\end{equation}

\begin{equation}
    C(T) = - \frac{\partial}{\partial T} E(T) 
        = \frac{1}{T^2 Z(T)} \sum_{i=1}^{N_s} |E_i|^2 e^{-\beta E_i} 
            - \frac{|E(T)|^2}{T^2}
\end{equation}

\begin{equation}
    S(T) = \frac{E(T)}{T} + \ln Z(T)
\end{equation}

The $z$-component of SSSF is:

\begin{align}
    & S_q^z  = \frac{1}{N} \sum_{j,k} e^{i {\bf q} \cdot ({\bf r}_j - {\bf r}_k)}
    S_{{\bf r}_j}^z S_{{\bf r}_k}^z \\
   & S_q^z (T) = <S_q^z>(T) = \frac{1}{Z(T)} {\rm Tr} (S_q^z e^{-\beta H})
       = \frac{1}{Z(T)} \sum_{i=1}^{N_s} \langle \psi_i | S_q^z | \psi_i \rangle e^{-\beta E_i} 
\end{align}


In [ ]:
# Thermaldynamic quanties
